EXEMPLO - TABELA COM METADADOS

In [1]:
import sys
!{sys.executable} -m pip install pyarrow

    100% |████████████████████████████████| 31.6MB 21kB/s  eta 0:00:01
You are using pip version 9.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import s3fs
import pyarrow.parquet as pq
from pyarrow.filesystem import S3FSWrapper

fs = s3fs.S3FileSystem()
bucket = "treinamento-big-data-aws"
path = "Turma_20190719/Gustavo/etl_application_train_v000.parquet"

# Python 3.6 or later
p_dataset = pq.ParquetDataset(
    f"s3://{bucket}/{path}",
    filesystem=fs)
df_app_train = p_dataset.read().to_pandas()

df_app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# Vamos renomear os targets e id
df_app_train.rename(columns={'SK_ID_CURR': 'id',
                            'TARGET': 'target'},
                    inplace=True)

In [21]:
 # Vamos verificar as variáveis e os respectivos tipos
df_app_train.dtypes   

id                              int32
target                          int32
NAME_CONTRACT_TYPE             object
CODE_GENDER                    object
FLAG_OWN_CAR                   object
FLAG_OWN_REALTY                object
CNT_CHILDREN                    int32
AMT_INCOME_TOTAL              float64
AMT_CREDIT                    float64
AMT_ANNUITY                   float64
AMT_GOODS_PRICE               float64
NAME_TYPE_SUITE                object
NAME_INCOME_TYPE               object
NAME_EDUCATION_TYPE            object
NAME_FAMILY_STATUS             object
NAME_HOUSING_TYPE              object
REGION_POPULATION_RELATIVE    float64
DAYS_BIRTH                      int32
DAYS_EMPLOYED                   int32
DAYS_REGISTRATION             float64
DAYS_ID_PUBLISH                 int32
OWN_CAR_AGE                   float64
FLAG_MOBIL                      int32
FLAG_EMP_PHONE                  int32
FLAG_WORK_PHONE                 int32
FLAG_CONT_MOBILE                int32
FLAG_PHONE  

In [22]:
import pandas as pd

def AjusteMetadados(dataframe): 

    train = dataframe
    # Verifica os tipos de variáveis presentes na tabela de treino
    t = []
    for i in train.columns:
            t.append(train[i].dtype)

    n = []
    for i in train.columns:
            n.append(i)

    aux_t = pd.DataFrame(data=t,columns=["Tipos"])
    aux_n = pd.DataFrame(data=n,columns=["Features"])
    df_tipovars = pd.concat([aux_n, aux_t], axis=1, join_axes=[aux_n.index])

    data = []
    for f in train.columns:

        # Definindo o papel das variáveis:
        if f == 'target':
            role = 'target'
        elif f == 'id':
            role = 'id'
        else:
            role = 'input'

        # Definindo o tipo das variáveis: nominal, ordinal, binary ou interval
        if f == 'target':
            level = 'binary'
        if train[f].dtype == 'object' or f == 'id': 
            level = 'nominal'
        elif train[f].dtype in ['float','float64'] :
            level = 'interval'
        elif train[f].dtype in ['int','int64','int32'] :
            level = 'ordinal'
        else:
            level = 'NA'

        # Todas variáveis são incializadas com keep exceto o id
        keep = True
        if f == 'id':
            keep = False

        # Definindo o tipo das variáveis da tabela de entrada
        dtype = train[f].dtype

        # Criando a lista com todo metadados
        f_dict = {
            'Features': f,
            'Role': role,
            'Level': level,
            'Keep': keep,
            'Tipo': dtype
        }
        data.append(f_dict)

    meta = pd.DataFrame(data, columns=['Features', 'Role', 'Level', 'Keep', 'Tipo'])

    # Quantidade de domínios distintos para cada cariável do tipo ordinal e nominal
    card = []

    v = train.columns
    for f in v:
        dist_values = train[f].value_counts().shape[0]
        f_dict = {
                'Features': f,
                'Cardinality': dist_values
            }
        card.append(f_dict)

    card = pd.DataFrame(card, columns=['Features', 'Cardinality'])

    metadados_train = pd.merge(meta, card, on='Features')

    return metadados_train 

In [23]:
metadados_app_train = AjusteMetadados(df_app_train)
metadados_app_train

,Features,Role,Level,Keep,Tipo,Cardinality
0,id,id,nominal,False,int32,307511
1,target,target,ordinal,True,int32,2
2,NAME_CONTRACT_TYPE,input,nominal,True,object,2
3,CODE_GENDER,input,nominal,True,object,3
4,FLAG_OWN_CAR,input,nominal,True,object,2
5,FLAG_OWN_REALTY,input,nominal,True,object,2
6,CNT_CHILDREN,input,ordinal,True,int32,15
7,AMT_INCOME_TOTAL,input,interval,True,float64,2548
8,AMT_CREDIT,input,interval,True,float64,5603
9,AMT_ANNUITY,input,interval,True,float64,13672


In [27]:
metadados_app_train.to_csv('metadados_app_train.csv',
                           sep=';',
                           encoding='utf-8',
                           index=True)